# Loading Medicare and Medicaid Claims data into i2b2

focus is currently on carrier claims

(demographics was done in Oracle PL/SQL)

so far, we can get data in chunks, map patients and encounters, pivot diagnoses, and insert the result into an observation_fact table (which is missing some constraints).


## Python Data Science Tools

especially [pandas](http://pandas.pydata.org/pandas-docs/)

In [ ]:
import pandas as pd
import numpy as np
import sqlalchemy as sqla
dict(pandas=pd.__version__, numpy=np.__version__, sqlalchemy=sqla.__version__)

## Data Access via Luigi Config

[luigi docs](https://luigi.readthedocs.io/en/stable/)

In [ ]:
import luigi

In [ ]:
import pathlib

In [ ]:
def explicit_config(configRd):
    cls = luigi.configuration.LuigiConfigParser
    cls._instance = None  # KLUDGE
    cls._config_paths = [str(configRd)]
    return cls.instance()

explicit_config(pathlib.Path('luigi-sgrouse.cfg'))
luigi.configuration.LuigiConfigParser.instance()._config_paths

In [ ]:
import etl_tasks
import cms_etl
from script_lib import Script

In [ ]:
class CarrierClaims(etl_tasks.DBAccessTask, cms_etl.FromCMS):
    table = 'CMS_DEID.bcarrier_claims'

    def chunk(self, bene_id_first, bene_id_last):
        # TODO: use sqlalchemy API
        q = '''
        select *
        from %(T)s
        where bene_id between :bene_id_first and :bene_id_last
        ''' % dict(T=self.table)
        with self.connection() as conn:
            t = etl_tasks.SqlScriptTask(script=Script.cms_facts_load)
            plan = t.explain_plan(conn, q)
            print('@@', '\n'.join(plan))
            return pd.read_sql(q, conn._conn,
                               params=dict(bene_id_first=bene_id_first,
                                           bene_id_last=bene_id_last))

cc = CarrierClaims()
cc.account, cc.source.cms_rif

In [ ]:
# TODO: parameterize CMS_DEID
def bene_id_survey(conn, source_table, schema,
                   chunk_qty=100000,
                   parallel_degree=10):
    q = '''
      select chunk_num
        , count(*) chunk_size
        , min(bene_id) bene_id_first
        , max(bene_id) bene_id_last
        from (
        select bene_id, ntile(:chunk_qty) over (order by bene_id) as chunk_num
        from (select /*+ parallel(%(DEGREE)d) */ distinct bene_id
              from %(SCHEMA)s.%(SOURCE_TABLE)s
              /* Eliminate null case so that index can be used. */
              where bene_id is not null)
        ) group by chunk_num
      order by chunk_num  
    ''' % dict(SOURCE_TABLE=source_table, SCHEMA=schema, DEGREE=parallel_degree)
    return pd.read_sql(q, conn, params=dict(chunk_qty=chunk_qty)).set_index('chunk_num')

with cc.connection() as conn:
    bene_chunks = bene_id_survey(conn._conn, 'bcarrier_claims', cc.source.cms_rif)

bene_chunks.head()

In [ ]:
with cc.connection() as conn:
    data = cc.chunk(bene_chunks.iloc[0].bene_id_first,
                    bene_chunks.iloc[0].bene_id_last)
data.info()

In [ ]:
data.head()

## Column Info: Level of Measurement

Assign [levels of measurement](https://en.wikipedia.org/wiki/Level_of_measurement) based on column info:

In [ ]:
def column_measurement(col_info):
    m = col_info[['column_name', 'data_type']].copy()
    m['valtype_cd'] = '@'
    m.loc[col_info.data_type == 'NUMBER', 'valtype_cd'] = 'n'
    m.loc[col_info.data_type == 'DATE', 'valtype_cd'] = 'd'
    for text_id_suffix in ['_NPI', '_UPIN', '_TRIL_NUM', '_PIN_NUM']:
        m.loc[col_info.column_name.str.endswith(text_id_suffix), 'valtype_cd'] = 't'
    m.sort_index(inplace=True)
    return m

with cc.connection() as lc:
    q = '''
      select table_name, column_id, column_name, data_type
      from all_tab_columns
      where owner = :cms_rif
        and table_name not like 'SYS_%'
    '''
    information_schema = pd.read_sql(q, lc._conn, params=dict(cms_rif=cc.source.cms_rif)).set_index('column_id')


bcarrier_cols = information_schema[information_schema.table_name == 'BCARRIER_CLAIMS'].copy()
bcarrier_cols['dx'] = bcarrier_cols.column_name.str.contains('_DGNS_')
column_measurement(bcarrier_cols[~ bcarrier_cols.dx])

## Nominal Data

In [ ]:
key_cols = ['bene_id', 'clm_id', 'clm_from_dt', 'clm_thru_dt']
nominal_cols = column_measurement(bcarrier_cols[~ bcarrier_cols.dx &
                                               ~bcarrier_cols.column_name.str.lower().isin(key_cols)])
nominal_cols = nominal_cols[nominal_cols.valtype_cd == '@']
nominal_cols

In [ ]:
nominal_data = data[key_cols + [n for n in nominal_cols.column_name.str.lower()]].set_index(key_cols)
nominal_data.head()

### carr_num - coded, not numeric

It looks like numeric info, but [carr_num docs](https://www.resdac.org/cms-data/variables/Carrier-Number) say it's a code.

In [ ]:
### AMBIENT
def resdac_pg(path):
    import urllib.request
    addr = 'https://www.resdac.org/sites/resdac.umn.edu/' + path
    content = urllib.request.urlopen(addr).read()
    return content.decode('utf-8')

pg = resdac_pg('files/Carrier%20Number-MAC%20Table.txt')
len(pg)

In [ ]:
def carr_num_table_line(txt):
    code, label = txt.split(' = ', 1)
    if '(' in label:
        label, note = label.split(' (', 1)
        note = '(' + note
    else:
        note = None
    return dict(code=code, label=label, note=note)

def carr_num_db(text):
    lines = text.split('\r\n')
    return pd.DataFrame([carr_num_table_line(l)
                         for l in lines if ' = ' in l])

carr_num_db(pg).head()

In [ ]:
obs_cd = (nominal_data.reset_index().melt(id_vars=key_cols, var_name='column')
          .dropna(subset=['value'])
          .sort_values(['clm_id', 'column'])
          .set_index(key_cols))

obs_cd['valtype_cd'] = '@'
obs_cd['concept_cd'] = obs_cd.column.str.upper() + ':' + obs_cd.value
obs_cd.head(20)

### Diagnoses from Carrier Claims

In [ ]:
dx_cols = column_measurement(bcarrier_cols[bcarrier_cols.dx])
dx_vrsn_cols = dx_cols[1::2].reset_index()[['column_name']]
dx_cols = dx_cols[::2].reset_index()[['column_name']]
dx_cols = dx_cols.merge(dx_vrsn_cols, left_index=True, right_index=True, suffixes=['_cd', '_vrsn'])
dx_cols

In [ ]:
def fmt_dx_code(dgns_vrsn, dgns_cd) -> str:
    if dgns_vrsn == '10':
        return 'ICD10:' + row.dgns_cd  # TODO: ICD10 formatting
    # was: when dgns_vrsn = '9'
    #   but I found null dgns_vrsn e.g. one record with ADMTG_DGNS_CD = V5789
    #   so let's default to the IDC9 case
    return 'ICD9:' + dgns_cd[:3] + (
        ('.' + dgns_cd[3:]) if len(dgns_cd) > 3
        else '')


def dx_stack(data, dx_cols, key_cols):
    out = None
    for ix, pair in dx_cols.iterrows():
        dx_data = data[key_cols + [pair.column_name_vrsn.lower(),
                                   pair.column_name_cd.lower()]].set_index(key_cols)
        # icd_dgns_cd11 -> icd_dgns_cd
        dx_data.columns = ['dgns_vrsn', 'dgns_cd']
        dx_data = dx_data.dropna(subset=['dgns_cd'])
        dx_data['ix'] = ix
        dx_data['column'] = pair.column_name_cd.lower()
        if out is None:
            out = dx_data
        else:
            out = out.append(dx_data)
    return out

dx_data = dx_stack(data, dx_cols, key_cols)
dx_data['valtype_cd'] = '@'
dx_data['concept_cd'] = [fmt_dx_code(row.dgns_vrsn, row.dgns_cd)
                         for _, row in dx_data.iterrows()]
dx_data.reset_index().sort_values(['clm_id', 'ix']).set_index(key_cols).head(40)

## Patient, Encounter Mapping

In [ ]:
class BeneMapper(object):
    key_cols = '(BENE_ID)'

    def __init__(self, patient_mapping):
        self.patient_mapping = patient_mapping

    @classmethod
    def make(cls, conn, i2b2star, source_cd,
             bene_id_first, bene_id_last):
        q = '''
        select patient_ide bene_id, patient_num
        from %(I2B2STAR)s.patient_mapping
        where patient_ide_source = :patient_ide_source
        and patient_ide between :bene_id_first and :bene_id_last
        ''' % dict(I2B2STAR=i2b2star)
        
        patient_mapping = pd.read_sql(
            q, conn, params=dict(patient_ide_source=source_cd[1:-1] + cls.key_cols,
                                 bene_id_first=bene_id_first,
                                 bene_id_last=bene_id_last))
        return cls(patient_mapping)

    def lookup(self, data):
        return data.merge(self.patient_mapping, on='bene_id')

with cc.connection() as lc:
    bm = BeneMapper.make(lc._conn, cc.project.star_schema, cc.source.source_cd,
                        bene_chunks.iloc[0].bene_id_first, bene_chunks.iloc[0].bene_id_last)
bm.patient_mapping.info()

In [ ]:
bm.patient_mapping.head()

In [ ]:
bm.lookup(dx_data.reset_index().head())

In [ ]:
class MedparMapper(object):
    key_cols = '(MEDPAR_ID)'

    def __init__(self, encounter_mapping):
        self.encounter_mapping = encounter_mapping

    @classmethod
    def make(cls, conn, i2b2star, source_cd,
             bene_id_first, bene_id_last):
        q = '''
        select encounter_ide medpar_id, encounter_num
        from %(I2B2STAR)s.encounter_mapping
        where encounter_ide_source = :encounter_ide_source
        and patient_ide between :bene_id_first and :bene_id_last
        ''' % dict(I2B2STAR=i2b2star)
        
        encounter_mapping = pd.read_sql(
            q, conn, params=dict(encounter_ide_source=source_cd[1:-1] + cls.key_cols,
                                 bene_id_first=bene_id_first,
                                 bene_id_last=bene_id_last))
        return cls(encounter_mapping)

    @classmethod
    def fmt_patient_day(cls, df):
        return df.start_date.dt.strftime('%Y-%m-%d') + ' ' + df.bene_id

    @classmethod
    def _fallback(cls, df):
        # @@TODO: replace hash with something portable between Oracle and python
        return - cls.fmt_patient_day(df).apply(hash).abs()

    def lookup(self, data):
        x = data.merge(self.encounter_mapping, on='medpar_id', how='left')
        x.encounter_num = x.encounter_num.fillna(self._fallback(x))
        return x

In [ ]:
with cc.connection() as lc:
    mm0 = MedparMapper.make(lc._conn, cc.project.star_schema, cc.source.source_cd,
                        bene_chunks.iloc[0].bene_id_first, bene_chunks.iloc[0].bene_id_last)
mm0.encounter_mapping.info()

In [ ]:
mm = MedparMapper(mm0.encounter_mapping)
x = dx_data.head(50).reset_index().rename(columns=dict(clm_from_dt='start_date', clm_thru_dt='end_date'))
x['medpar_id'] = np.nan

mm.lookup(bm.lookup(x)).sort_values('start_date')

In [ ]:
i2b2_key_cols = ['patient_num', 'encounter_num', 'start_date', 'concept_cd']
i2b2_cols = i2b2_key_cols + ['valtype_cd', 'end_date']

obs_fact = mm.lookup(bm.lookup(x))[i2b2_cols]
obs_fact['provider_id'] = '@'  # TODO
obs_fact['modifier_cd'] = bcarrier_cols.table_name.iloc[0]
obs_fact['instance_num'] = obs_fact.index
obs_fact.head()

In [ ]:
with cc.connection() as lc:
    obs_fact.to_sql(name='observation_fact_100', con=lc._conn,
                   if_exists='append', index=False)